# What is Langchain?

Langchain is a Python framework for building and working with LLM APIs and related tools. It is designed to be easy to use and flexible. It introduces two new abstractions: chains and agents. Chains are "invoked", while agents are "executed". 

The framework has some bells and whistles: Langchain Expression Language (LCEL), used to describe the chains and inputs to it. And a paid service for request tracing called Langsmith.

This notebook is based on [Langchain Quickstart](https://python.langchain.com/docs/use_cases/question_answering/quickstart/)

Langchain is secretly not a single Python package, but shipped as a collection of packages:

1. Core: Parsers, Runnables
2. Community: Vector Stores, Document Loaders
3. Text Splitters, LLM APIs, and more

The framework is designed to be powerful at the cost of being complex. It has happy paths for beginners, but primarily built with a lot of advanced features to enable experimentation. It is not a "plug and play" framework, but a "plug and experiment" one.

In [1]:
import getpass
import os

import bs4
from langchain import hub

## Langchain imports
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Qdrant
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [4]:
llm = ChatOpenAI(model="gpt-4-turbo")

## Getting the data

This is web scraping logic with a BeautifulSoup parser. BS4 is not a part of Langchain, but it is a popular library for parsing HTML and XML documents. The code below is a simple example of how to scrape a website for data.

In [7]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [14]:
docs[0].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}

In [17]:
chars = len(docs[0].page_content)
approx_tokens = chars*3/4
approx_tokens

32348.25

> 💡 Reader Exercise: Find the exact number of tokens for GPT4 model instead of approximate tokens. Post your code snippet on Discourse!

## Chunking

Chunking is a process of breaking down a text into smaller parts. 

In [30]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)

In [56]:
splits[:5]

[Document(page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n

In [33]:
vectorstore = Qdrant.from_documents(documents=splits, embedding=OpenAIEmbeddings(), output_parser=StrOutputParser(), collection_name="lilianweng", location=":memory:")

2024-04-12 06:55:32.302 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


In [35]:
retriever = vectorstore.as_retriever()

In [37]:
prompt = hub.pull("rlm/rag-prompt")

In [47]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [48]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [52]:
rag_chain.invoke("What do we about planning and subgoals?")

'In planning and subgoals, tasks are decomposed into manageable steps or subgoals to simplify complex problems and enhance task execution. Techniques such as Chain of Thought (CoT) and Tree of Thoughts (ToT) enable an agent to think step-by-step through a problem, exploring multiple reasoning paths. Additionally, external tools like classical planners can be used for long-horizon planning, interfacing through Planning Domain Definition Language (PDDL) to translate and execute plans.'